# DeepMSim: A Deep Learning based MS/MS spectra Simulator
### *Muhammad Haseeb and Sumesh Kumar*
##### *CAP5610: Machine Learning - Fall 2019*
##### *School of Computing and Information Sciences*
##### *Florida International University (FIU)*

# Licence Information

In [5]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [7]:
# The MIT License

# Copyright 2019 Muhammad Haseeb and Sumesh Kumar

# Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
# and associated documentation files (the "Software"), to deal in the Software without restriction, 
# including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
# and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
# subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial 
# portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
# LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
# IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
# WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
# SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Transformer model for MS/MS simulation
We will implement a <a href="https://arxiv.org/abs/1706.03762" class="external">Transformer model</a> to translate the peptide sequences into spectral peak sequences using TensorFlow.

Transformer model employs the concept of *self-attention* — the ability to attend to different positions of the input sequence to compute a representation of that sequence for modelling. It is composed of several layer multiconnected encoders and decoders that construct the output sequences. 

However, the drawback is that if the input sequences have a *temporal/spatial* (positional) relationships such as in peptide sequence data, a positional encoding such as <a href="https://arxiv.org/abs/1810.04805BERT" class="external">BERT encoding</a> must be added or the transformers will only see the input sequences as bag {set} of unordered and unrelated words. 

## Import the required libraries

In [47]:
# Required Imports
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import os
import re
import sys
import math
import time
import urllib
import numpy as np
import matplotlib.pyplot as plt

# Data Acquisition
Let's get the annotated MS/MS spectral data libraries from <a href="https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload" class="external">NIST.</a>

**CAUTION:** Make sure the downloaded spectral library datasets have been sourced from the same Mass-spectrometer instrument using the same fragmentation method.


In [28]:
# Getting the Mouse Orbitrap spectral library (.MSP format) from NIST and placing it in the speclib folder
nist_url = 'ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz'

data_dir = os.path.expanduser("./speclib")
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

!wget 'ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz'

--2019-11-24 22:26:09--  ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz
           => ‘cptac2_mouse_hcd_selected.msp.tar.gz’
Resolving chemdata.nist.gov (chemdata.nist.gov)... 129.6.24.33, 2610:20:6005:24::33
Connecting to chemdata.nist.gov (chemdata.nist.gov)|129.6.24.33|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /download/peptide_library/libraries/cptaclib/2015 ... done.
==> SIZE cptac2_mouse_hcd_selected.msp.tar.gz ... 61660818
==> PASV ... done.    ==> RETR cptac2_mouse_hcd_selected.msp.tar.gz ... done.
Length: 61660818 (59M) (unauthoritative)

cptac2_mouse_hcd_se 100%[===================>]  58.80M  21.0MB/s    in 2.8s    

2019-11-24 22:26:12 (21.0 MB/s) - ‘cptac2_mouse_hcd_selected.msp.tar.gz’ saved [61660818]



In [36]:
# Extract the tar ball into MSP file and move into the speclib folder
msp_file = './speclib/cptac2_mouse_hcd_selected.msp'

!tar xvf 'cptac2_mouse_hcd_selected.msp.tar.gz'
os.rename('./cptac2_mouse_hcd_selected.msp', msp_file)

cptac2_mouse_hcd_selected.msp


# Data Extraction and Preprocessing

Mine the spectral libraries and extract corpuses for both languages. i.e. peptide sequences and their MS/MS spectra.
Construct a dataframe with sentences and their translations, with words separated by a '*space* and ending with ' .' for both languages. Then, Convert the dataframe to a tensor dataset.

## Process known Immonium Ions
The commonly observed immonium ions along with their isotopes are added to a dictionary un order to represent the ion sentences (from the corpus) in terms of the dictionary (bag of words)

In [119]:
# Known immonium ions
imm = np.array(['AA', 'RA', 'RB', 'RC', 'RD', 'RE', 'RF', 'RG', 'RJ', 'NA', 'NB', 'DA', 'DB', 'CA', 'EA', 'QA', 'QB',
                'QC', 'QD', 'GA', 'HA', 'HB', 'HC', 'HD', 'HE', 'HF', 'IA', 'IB', 'IC', 'LA', 'LB', 'LC', 'KA', 'KB',
                'KC', 'KD', 'KE', 'KF', 'MA', 'MB', 'FA', 'FB', 'PA', 'SA', 'TA', 'WA', 'WB', 'WC', 'WD', 'WE', 'WF',
                'WG', 'YA', 'YB', 'YC', 'VA', 'VB', 'VC', 'VD'])

# Make isotopes of an array of ions
iso = []

# For each immonium ion, append the +i and +2i isotopes
for im in imm:
    iso.append(im + '+i')
    iso.append(im + '+2i')

# Append the isotopes to the main dictionary
imm = np.append(imm, np.array(iso))

# Make a set representation
simm= set(imm)

## Corpus Extraction
Extract the peptide sequence and ion sentence sequences from the dataset.

### MSP file parser

In [172]:
# Create empty lists to store temporary data
seq = []
X = []
Z = []
E = []

# Parse the MSP file and get the immonium ion dataset
def parseMSPfile(fname=None, dimm=None):
    header = False
    stnc = []
    with open(fname, 'r') as slib:
        for line in slib:
            # Check for empty line for spectrum end and header start
            if (line[0] == '\r' or line[0] == '#' or line[0] == '\n'):
                header = False
                X.append(stnc[:])
                stnc.clear()
                continue

            # Check if spectrum header
            if (header == False):
                if (line[:7] != 'Comment'):
                    key, val = line.split(':')

                    if key == 'Name':
                        # Take out the sequence
                        nm, chg = val.split('/')
                        seq.append(nm[1:])
                        # Take out the charge, nmods and eV (energy)
                        ch = chg.split('_')

                        # Append the charge
                        Z.append(int(ch[0]))
                        
                        # Append the energy
                        #print(ch)
                        E.append(float(ch[-1][:-3]))
                        

                    if key == 'Num peaks':
                        header = True

            # If not the header, then it is the peak list
            else:
                _, _, label = line.split('\t')

                # Clip off the first and last " "
                label = label[1:-1]

                # Extract label pattern
                lbls = re.findall(r"(?<!\/)(?<![A-Z])(I[A-Z][A-Z](?![A-Z]){2}[+]*\d*i*)", label)
                lbls = list(map(lambda x: x[1:], lbls))

                # If anything to append then append
                if lbls:
                    for lab in lbls:
                        if lab not in dimm:
                            dimm.add(lab[0])
                            #print ('Label Added:' + lab)
                    #print(lbls)
                    stnc.extend(lbls)

    # For the last one
    X.append(stnc[:])
    stnc.clear()

In [181]:
# Clear the lists
seq = []
X = []
Z = []
E = []

# Parse the MSP file and create a dataframe
_ = parseMSPfile(fname = msp_file, dimm = simm)

### Construct the two languages

In [194]:
# Variables
MAXZ = 3

In [195]:
# Construct a dataframe
dataset = pd.DataFrame(list(zip(seq, Z, E, X)), columns=['seq', 'z', 'E', 'ions'])

# Filter the dataset by the max charge 
dataset = dataset[dataset['z'] <= MAXZ]

# Print a few entries to visualize
print(dataset[:20])

                                  seq  z     E  \
0           AAAAAAAAAAEEAAMQRDLLPPAGR  3  34.9   
1                          AAADDGEEPK  2  24.4   
2      AAAEEGCGVGVEDDRELEELLESALDDFDK  3  50.2   
3                      AAAEVNQEYGLDPK  2  31.0   
4                     AAAFEDQENETVVVK  2  38.7   
5                     AAAFEDQENETVVVK  2  40.1   
6                   AAAGELQEDSGLHVLAR  3  19.0   
7                      AAAITSDLLESLGR  2  29.0   
8                      AAAITSDLLESLGR  2  33.2   
9                      AAAITSDLLESLGR  2  44.0   
10                          AAALERMPR  2  21.2   
11                          AAATLMTER  2  22.6   
12              AAATQPDGKDTPDEPWAFPAR  3  34.3   
13  AACEQLHQQQQQQQEETTAATLLLQGEEEGEED  3  57.7   
14                            AACFQLK  2  19.6   
15                  AACLESAQEPAGAWSNK  2  41.9   
16                  AACLESAQEPAGAWSNK  2  43.5   
17                          AADAVEDLR  2  30.0   
19                           AADIPGLK  2  18.4   


In [218]:
lang1 = list(map(lambda x: " ".join(x) + ' .', dataset['seq']))
lang2 = list(map(lambda y: " ".join(y) + ' .', dataset['ions']))

# Clear the duplicate data
seq.clear()
X.clear()
Z.clear()
E.clear()

['A A A A A A A A A A E E A A M Q R D L L P P A G R .', 'A A A D D G E E P K .', 'A A A E E G C G V G V E D D R E L E E L L E S A L D D F D K .', 'A A A E V N Q E Y G L D P K .', 'A A A F E D Q E N E T V V V K .', 'A A A F E D Q E N E T V V V K .', 'A A A G E L Q E D S G L H V L A R .', 'A A A I T S D L L E S L G R .', 'A A A I T S D L L E S L G R .', 'A A A I T S D L L E S L G R .']


In [230]:
# Insert the languages back to the dataframe
dataset['seq'] = lang1
dataset['ions'] = lang2

# Language Encoding

Tokenize the sentences, construct tensors, embed the charge and eV information and do positional encoding (BERT).

In [235]:
# Print the head of dataset
print(dataset[:20])

                                                  seq  z     E  \
0   A A A A A A A A A A E E A A M Q R D L L P P A ...  3  34.9   
1                               A A A D D G E E P K .  2  24.4   
2   A A A E E G C G V G V E D D R E L E E L L E S ...  3  50.2   
3                       A A A E V N Q E Y G L D P K .  2  31.0   
4                     A A A F E D Q E N E T V V V K .  2  38.7   
5                     A A A F E D Q E N E T V V V K .  2  40.1   
6                 A A A G E L Q E D S G L H V L A R .  3  19.0   
7                       A A A I T S D L L E S L G R .  2  29.0   
8                       A A A I T S D L L E S L G R .  2  33.2   
9                       A A A I T S D L L E S L G R .  2  44.0   
10                                A A A L E R M P R .  2  21.2   
11                                A A A T L M T E R .  2  22.6   
12        A A A T Q P D G K D T P D E P W A F P A R .  3  34.3   
13  A A C E Q L H Q Q Q Q Q Q Q E E T T A A T L L ...  3  57.7   
14        